# Introduction

QA plots for the INTT detector

In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

In [2]:
# turn off/on code for the result HTML page

display(Markdown('*For the result HTML page:* '))
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('div.jp-CodeMirrorEditor').hide(); // for newer versions of nbviewer.org
 } else {
 $('div.input').show();
 $('div.jp-CodeMirrorEditor').show(); // for newer versions of nbviewer.org
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

*For the result HTML page:* 

In [3]:
import os.path

# readme file of the macros, available if run under JenkinsCI
# https://github.com/sPHENIX-Collaboration/utilities/blob/master/jenkins/built-test/test-tracking-qa.sh 

macro_markdown = 'Fun4All-macros-README.md'

if os.path.isfile(macro_markdown) :
    with open(macro_markdown, 'r') as file:
        display(Markdown(file.read()))

## `pyROOT` env check

In [4]:
import ROOT

OFFLINE_MAIN = os.getenv("OFFLINE_MAIN")
if OFFLINE_MAIN is not None:
    display(Markdown(f"via sPHENIX software distribution at `{OFFLINE_MAIN}`"))

via sPHENIX software distribution at `/cvmfs/sphenix.sdcc.bnl.gov/alma9.2-gcc-14.2.0/release/release_ana/ana.471`

## Plotting source code

In [5]:
import subprocess

try:
    git_url = \
        subprocess.run(['git','remote','get-url','origin'], stdout=subprocess.PIPE)\
        .stdout.decode('utf-8').strip()\
        .replace('git@github.com:','https://github.com/')

    display(Markdown(f"View the source code repository at {git_url}"))
except: # catch *all* exceptions
    #     well do nothing
    pass

View the source code repository at https://github.com/sPHENIX-Collaboration/QA-gallery.git

## JenkinsCI information (if available)

In [6]:
display(Markdown('Some further details about the QA run, if executed under the Jenkins CI:'))

checkrun_repo_commit = os.getenv("checkrun_repo_commit")
if checkrun_repo_commit is not None:
    display(Markdown(f"* The commit being checked is {checkrun_repo_commit}"))
        
ghprbPullLink = os.getenv("ghprbPullLink")
if ghprbPullLink is not None:
    display(Markdown(f"* Link to the pull request: {ghprbPullLink}"))

BUILD_URL =  os.getenv("BUILD_URL")
if BUILD_URL is not None:
    display(Markdown(f"* Link to the build: {BUILD_URL}"))

git_url_macros =  os.getenv("git_url_macros")
sha_macros =  os.getenv("sha_macros")
if git_url_macros is not None:
    display(Markdown(f"* Git repo for macros: {git_url_macros} , which merges `{sha_macros}` and the QA tracking branch"))

RUN_ARTIFACTS_DISPLAY_URL = os.getenv("RUN_ARTIFACTS_DISPLAY_URL")
if RUN_ARTIFACTS_DISPLAY_URL is not None:
    display(Markdown(f"* Download the QA ROOT files: {RUN_ARTIFACTS_DISPLAY_URL}"))

JENKINS_URL  = os.getenv("JENKINS_URL")
if JENKINS_URL is not None:
    display(Markdown(f"Automatically generated by [sPHENIX Jenkins continuous integration]({JENKINS_URL}) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)"))

Some further details about the QA run, if executed under the Jenkins CI:


# Initialization

In [7]:
%%cpp -d

#include "QA_Draw_Utility.C"

#include <sPhenixStyle.C>

#include <TFile.h>
#include <TLine.h>
#include <TString.h>
#include <TTree.h>
#include <cassert>
#include <cmath>
//#include <boost/format.hpp>

In [8]:
%%cpp

SetsPhenixStyle();
TVirtualFitter::SetDefaultFitter("Minuit2");

// test sPHENIX lib load
// gSystem->Load("libg4eval.so");

// test libs
// gSystem->ListLibraries();

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [9]:
%jsroot on

In [10]:

qa_file_name_new = os.getenv("qa_file_name_new")
if qa_file_name_new is None:
    qa_file_name_new = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_new` env not set. use the default `qa_file_name_new={qa_file_name_new}`"))


`qa_file_name_new` env not set. use the default `qa_file_name_new=prdf_reconstruction53877_qa.root`

In [11]:
# qa_file_new = ROOT.TFile.Open(qa_file_name_new);

# assert qa_file_new.IsOpen()
# qa_file_new.ls()
display(Markdown(f"Openning QA file at `{qa_file_name_new}`"))
ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_new = new TFile(\"{qa_file_name_new}\");")
ROOT.gInterpreter.ProcessLine(f"const char * qa_file_name_new = \"{qa_file_name_new}\";")

Openning QA file at `prdf_reconstruction53877_qa.root`

22862810345472

In [12]:
%%cpp

if (qa_file_new == nullptr) 
{
    cout <<"Error, can not open QA root file"<<qa_file_name_new<<endl;
    exit(1);
}
// list inputs histograms if needed
 //qa_file_new ->ls();

In [13]:
%%cpp

int TpcSeedsDrawTrackletInfo(TFile* qa_file_new) {
    std::cout << "Tpc Seeds DrawTrackletInfo() Beginning" << std::endl;
    
    // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error: Unable to open ROOT file." << std::endl;
        return -1;
    }

    std::string histprefix = "h_TpcSeedsQA_";

    // Define histograms
    TH1F* h_ntrack1d = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "nrecotracks1d").c_str()));
    TH1F* h_ntrack1d_pos = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "nrecotracks1d_pos").c_str()));
    TH1F* h_ntrack1d_neg = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "nrecotracks1d_neg").c_str()));
    TH1F* h_ntrack1d_ptg1 = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "nrecotracks1d_ptg1").c_str()));
    TH1F* h_ntrack1d_ptg1_pos = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "nrecotracks1d_ptg1_pos").c_str()));
    TH1F* h_ntrack1d_ptg1_neg = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "nrecotracks1d_ptg1_neg").c_str()));
    TH1F* h_pt = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "pt").c_str()));
    TH1F* h_pt_pos = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "pt_pos").c_str()));
    TH1F* h_pt_neg = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "pt_neg").c_str()));
    TH2F* h_ntrack_pos = dynamic_cast<TH2F*>(qa_file_new->Get((histprefix + "nrecotracks_pos").c_str()));
    TH2F* h_ntrack_neg = dynamic_cast<TH2F*>(qa_file_new->Get((histprefix + "nrecotracks_neg").c_str()));
    TH1F* h_ntrack_isfromvtx_pos = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "ntrack_isfromvtx_pos").c_str()));
    TH1F* h_ntrack_isfromvtx_neg = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "ntrack_isfromvtx_neg").c_str()));

    // Create the canvas and divide it into 3x3 grid
    TCanvas* canvas = new TCanvas("seed tracklet_info", "Seed Track Info", 1200, 900);
    canvas->Divide(3, 3);  // Dividing the canvas into 3x3 pads

    // First histogram pad: Number of TPC tracks
    canvas->cd(1);  // Selecting the first pad
    if (h_ntrack1d && h_ntrack1d_pos && h_ntrack1d_neg) {
        h_ntrack1d->SetTitle("Number of TPC tracks");
        h_ntrack1d->SetXTitle("Number of TPC tracks");
        h_ntrack1d->SetYTitle("Entries");
        h_ntrack1d->SetMarkerColor(kBlack);
        h_ntrack1d->SetLineColor(kBlack);
        auto ymax = h_ntrack1d->GetMaximum();
        auto ymax_pos = h_ntrack1d_pos->GetMaximum();
        auto ymax_neg = h_ntrack1d_neg->GetMaximum();
        h_ntrack1d->SetMaximum(1.2 * std::max({ymax, ymax_pos, ymax_neg}));
        h_ntrack1d->Draw("");
        h_ntrack1d_pos->SetMarkerColor(kRed);
        h_ntrack1d_pos->SetLineColor(kRed);
        h_ntrack1d_pos->Draw("same");
        h_ntrack1d_neg->SetMarkerColor(kBlue);
        h_ntrack1d_neg->SetLineColor(kBlue);
        h_ntrack1d_neg->Draw("same");
        auto legend = new TLegend(0.55, 0.7, 0.83, 0.9);
        legend->AddEntry(h_ntrack1d, "Positive + Negative", "pl");
        legend->AddEntry(h_ntrack1d_pos, "Positive charged", "pl");
        legend->AddEntry(h_ntrack1d_neg, "Negative charged", "pl");
        legend->Draw();
        gPad->SetRightMargin(0.15);
    } else {
        return -1;
    }

    // Second histogram pad: Number of TPC tracks with pT > 1GeV
    canvas->cd(2);  // Selecting the second pad
    canvas->SetLogy(1);
    if (h_ntrack1d_ptg1 && h_ntrack1d_ptg1_pos && h_ntrack1d_ptg1_neg) {
        h_ntrack1d_ptg1->SetTitle("Number of TPC tracks with p_{T} > 1GeV");
        h_ntrack1d_ptg1->SetXTitle("Number of TPC tracks");
        h_ntrack1d_ptg1->SetYTitle("Entries");
        h_ntrack1d_ptg1->SetMarkerColor(kBlack);
        h_ntrack1d_ptg1->SetLineColor(kBlack);
        auto ymax = h_ntrack1d_ptg1->GetMaximum();
        auto ymax_pos = h_ntrack1d_ptg1_pos->GetMaximum();
        auto ymax_neg = h_ntrack1d_ptg1_neg->GetMaximum();
        h_ntrack1d_ptg1->SetMaximum(1.2 * std::max({ymax, ymax_pos, ymax_neg}));
        h_ntrack1d_ptg1->Draw("");
        h_ntrack1d_ptg1_pos->SetMarkerColor(kRed);
        h_ntrack1d_ptg1_pos->SetLineColor(kRed);
        h_ntrack1d_ptg1_pos->Draw("same");
        h_ntrack1d_ptg1_neg->SetMarkerColor(kBlue);
        h_ntrack1d_ptg1_neg->SetLineColor(kBlue);
        h_ntrack1d_ptg1_neg->Draw("same");
        auto legend = new TLegend(0.55, 0.7, 0.83, 0.9);
        legend->AddEntry(h_ntrack1d_ptg1, "Positive + Negative", "pl");
        legend->AddEntry(h_ntrack1d_ptg1_pos, "Positive charged", "pl");
        legend->AddEntry(h_ntrack1d_ptg1_neg, "Negative charged", "pl");
        legend->Draw();
        gPad->SetRightMargin(0.15);
    } else {
        return -1;
    }

    // Third histogram pad: pT distribution
    canvas->cd(3);  // Selecting the third pad
    canvas->SetLogy(1);
    if (h_pt && h_pt_pos && h_pt_neg) {
        h_pt->SetTitle("p_{T} distribution");
        h_pt->SetXTitle("p_{T} [GeV]");
        h_pt->SetYTitle("Entries");
        h_pt->SetMarkerColor(kBlack);
        h_pt->SetLineColor(kBlack);
        auto ymax = h_pt->GetMaximum();
        auto ymax_pos = h_pt_pos->GetMaximum();
        auto ymax_neg = h_pt_neg->GetMaximum();
        h_pt->SetMaximum(1.2 * std::max({ymax, ymax_pos, ymax_neg}));
        h_pt->Draw("");
        h_pt_pos->SetMarkerColor(kRed);
        h_pt_pos->SetLineColor(kRed);
        h_pt_pos->Draw("same");
        h_pt_neg->SetMarkerColor(kBlue);
        h_pt_neg->SetLineColor(kBlue);
        h_pt_neg->Draw("same");
        auto legend = new TLegend(0.55, 0.7, 0.83, 0.9);
        legend->AddEntry(h_pt, "Positive + Negative", "pl");
        legend->AddEntry(h_pt_pos, "Positive charged", "pl");
        legend->AddEntry(h_pt_neg, "Negative charged", "pl");
        legend->Draw();
        gPad->SetRightMargin(0.15);
    } else {
        return -1;
    }

    // Fourth histogram pad: Track position (eta vs phi)
    canvas->cd(4);  // Selecting the fourth pad
    if (h_ntrack_pos) {
        h_ntrack_pos->SetXTitle("#eta");
        h_ntrack_pos->SetYTitle("#phi [rad]");
        h_ntrack_pos->SetZTitle("Number of positive tracks with p_{T} > 1GeV");
        h_ntrack_pos->Draw("colz");
        gPad->SetRightMargin(0.17);
    } else {
        return -1;
    }

    // Fifth histogram pad: Track position for negative tracks
    canvas->cd(5);  // Selecting the fifth pad
    if (h_ntrack_neg) {
        h_ntrack_neg->SetXTitle("#eta");
        h_ntrack_neg->SetYTitle("#phi [rad]");
        h_ntrack_neg->SetZTitle("Number of negative tracks with p_{T} > 1GeV");
        h_ntrack_neg->Draw("colz");
        gPad->SetRightMargin(0.17);
    } else {
        return -1;
    }

    // Sixth histogram pad: Positive track vertex association
    canvas->cd(6);  // Selecting the sixth pad
    if (h_ntrack_isfromvtx_pos) {
        h_ntrack_isfromvtx_pos->SetTitle("Is positive track from a vertex");
        h_ntrack_isfromvtx_pos->SetXTitle("Is positive track associated to a vertex");
        h_ntrack_isfromvtx_pos->SetYTitle("Normalized Entries");
        h_ntrack_isfromvtx_pos->Scale(1. / h_ntrack_isfromvtx_pos->Integral());
        h_ntrack_isfromvtx_pos->SetMarkerSize(1.5);
        h_ntrack_isfromvtx_pos->Draw("histtext0");
        gPad->SetRightMargin(0.15);
    } else {
        return -1;
    }

    // Seventh histogram pad: Negative track vertex association
    canvas->cd(7);  // Selecting the seventh pad
    if (h_ntrack_isfromvtx_neg) {
        h_ntrack_isfromvtx_neg->SetTitle("Is negative track from a vertex");
        h_ntrack_isfromvtx_neg->SetXTitle("Is negative track associated to a vertex");
        h_ntrack_isfromvtx_neg->SetYTitle("Normalized Entries");
        h_ntrack_isfromvtx_neg->Scale(1. / h_ntrack_isfromvtx_neg->Integral());
        h_ntrack_isfromvtx_neg->SetMarkerSize(1.5);
        h_ntrack_isfromvtx_neg->Draw("histtext0");
        gPad->SetRightMargin(0.15);
    } else {
        return -1;
    }

    // Add text to the canvas
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.03);
    PrintRun.SetNDC();
    PrintRun.SetTextAlign(23);  // center/top alignment
    std::ostringstream runnostream1;
    std::string runstring1;
    runstring1 = runnostream1.str();
    canvas->cd();
    PrintRun.DrawText(0.5, 1., runstring1.c_str());

    // Update the canvas
    canvas->Update();

    std::cout << "DrawTrackletInfo Ending" << std::endl;
    return 0;
}


In [14]:
%%cpp
TpcSeedsDrawTrackletInfo(qa_file_new)

Tpc Seeds DrawTrackletInfo() Beginning
DrawTrackletInfo Ending
(int) 0


In [15]:
#Draw Cluster Information 1

In [16]:
%%cpp -d

void DrawSeedClusterInfo1(TFile* qa_file_new) {
    std::cout << "Tpc Seeds DrawClusterInfo1() Beginning" << std::endl;
    
    // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error: Unable to open ROOT file." << std::endl;
        return;
    }

    std::string histprefix = "h_TpcSeedsQA_";


    // Retrieve histograms
    TH1F* h_ntpc_pos = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "ntpc_pos").c_str()));
    TH1F* h_ntpc_neg = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "ntpc_neg").c_str()));
    TH1F* h_ntpot_pos = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "ntpot_pos").c_str()));
    TH1F* h_ntpot_neg = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "ntpot_neg").c_str()));
    TProfile2D* h_avgnclus_eta_phi_pos = dynamic_cast<TProfile2D*>(qa_file_new->Get((histprefix + "avgnclus_eta_phi_pos").c_str()));
    TProfile2D* h_avgnclus_eta_phi_neg = dynamic_cast<TProfile2D*>(qa_file_new->Get((histprefix + "avgnclus_eta_phi_neg").c_str()));
    TH1F* h_clusphisize1frac_side0_0 = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "clusphisize1frac_side0_0").c_str()));
    TH1F* h_clusphisize1frac_side0_1 = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "clusphisize1frac_side0_1").c_str()));
    TH1F* h_clusphisize1frac_side0_2 = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "clusphisize1frac_side0_2").c_str()));
    TH1F* h_clusphisize1frac_side1_0 = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "clusphisize1frac_side1_0").c_str()));
    TH1F* h_clusphisize1frac_side1_1 = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "clusphisize1frac_side1_1").c_str()));
    TH1F* h_clusphisize1frac_side1_2 = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "clusphisize1frac_side1_2").c_str()));
    
    TCanvas* canvas = dynamic_cast<TCanvas*>(gROOT->FindObject("track_info"));
    if (!canvas) {
        canvas = new TCanvas("track_info", "Track Info", 1200, 900);
    } else {
    canvas->Clear();  // Clear the existing canvas before using it again
    }

    // Draw histograms on pads
    canvas->cd(1);  // Selecting the first pad
    if (h_ntpc_pos && h_ntpc_neg) {
        h_ntpc_pos->SetTitle("Number of TPC clusters for track with p_{T}>1GeV");
        h_ntpc_pos->SetXTitle("Number of TPC clusters");
        h_ntpc_pos->SetYTitle("Entries");
        h_ntpc_pos->SetMarkerColor(kRed);
        h_ntpc_pos->SetLineColor(kRed);
        auto ymax_pos = h_ntpc_pos->GetMaximum();
        auto ymax_neg = h_ntpc_neg->GetMaximum();
        h_ntpc_pos->SetMaximum(1.2 * std::max({ymax_pos, ymax_neg}));
        h_ntpc_pos->Draw("");
        h_ntpc_neg->SetMarkerColor(kBlue);
        h_ntpc_neg->SetLineColor(kBlue);
        h_ntpc_neg->Draw("same");
        auto legend = new TLegend(0.55, 0.7, 0.83, 0.9);
        legend->AddEntry(h_ntpc_pos, "Positive charged", "pl");
        legend->AddEntry(h_ntpc_neg, "Negative charged", "pl");
        legend->Draw();
        gPad->SetRightMargin(0.15);
    } else {
        std::cerr << "Histograms h_ntpc_pos or h_ntpc_neg not found" << std::endl;
        return;
    }

    canvas->cd(2);
    if (h_ntpot_pos && h_ntpot_neg) {
        h_ntpot_pos->SetTitle("Number of TPOT clusters for track with p_{T}>1GeV");
        h_ntpot_pos->SetXTitle("Number of TPOT clusters");
        h_ntpot_pos->SetYTitle("Entries");
        h_ntpot_pos->SetMarkerColor(kRed);
        h_ntpot_pos->SetLineColor(kRed);
        auto ymax_pos = h_ntpot_pos->GetMaximum();
        auto ymax_neg = h_ntpot_neg->GetMaximum();
        h_ntpot_pos->SetMaximum(1.2 * std::max({ymax_pos, ymax_neg}));
        h_ntpot_pos->Draw("");
        h_ntpot_neg->SetMarkerColor(kBlue);
        h_ntpot_neg->SetLineColor(kBlue);
        h_ntpot_neg->Draw("same");
        auto legend = new TLegend(0.55, 0.7, 0.83, 0.9);
        legend->AddEntry(h_ntpot_pos, "Positive charged", "pl");
        legend->AddEntry(h_ntpot_neg, "Negative charged", "pl");
        legend->Draw();
        gPad->SetRightMargin(0.15);
    } else {
        std::cerr << "Histograms h_ntpot_pos or h_ntpot_neg not found" << std::endl;
        return;
    }

    canvas->cd(3);
    if (h_avgnclus_eta_phi_pos) {
        h_avgnclus_eta_phi_pos->SetXTitle("#eta");
        h_avgnclus_eta_phi_pos->SetYTitle("#phi [rad]");
        h_avgnclus_eta_phi_pos->SetZTitle("Average number of clusters per positive track with p_{T}>1GeV");
        h_avgnclus_eta_phi_pos->Draw("colz");
        gPad->SetRightMargin(0.17);
    } else {
        std::cerr << "Histogram h_avgnclus_eta_phi_pos not found" << std::endl;
        return;
    }

    canvas->cd(4);
    if (h_avgnclus_eta_phi_neg) {
        h_avgnclus_eta_phi_neg->SetXTitle("#eta");
        h_avgnclus_eta_phi_neg->SetYTitle("#phi [rad]");
        h_avgnclus_eta_phi_neg->SetZTitle("Average number of clusters per negative track with p_{T}>1GeV");
        h_avgnclus_eta_phi_neg->Draw("colz");
        gPad->SetRightMargin(0.17);
    } else {
        std::cerr << "Histogram h_avgnclus_eta_phi_neg not found" << std::endl;
        return;
    }

    canvas->cd(5);
    if (h_clusphisize1frac_side0_0 && h_clusphisize1frac_side0_1 && h_clusphisize1frac_side0_2
        && h_clusphisize1frac_side1_0 && h_clusphisize1frac_side1_1 && h_clusphisize1frac_side1_2) {
        
        h_clusphisize1frac_side0_0->SetTitle("Fraction of TPC Cluster Phi Size == 1");
        h_clusphisize1frac_side0_0->SetXTitle("Fraction of TPC Cluster Phi Size == 1");
        h_clusphisize1frac_side0_0->SetYTitle("Entries");
        h_clusphisize1frac_side0_0->SetMarkerColor(kRed);
        h_clusphisize1frac_side0_0->SetLineColor(kRed);
        auto ymax_side0_0 = h_clusphisize1frac_side0_0->GetMaximum();
        auto ymax_side0_1 = h_clusphisize1frac_side0_1->GetMaximum();
        auto ymax_side0_2 = h_clusphisize1frac_side0_2->GetMaximum();
        auto ymax_side1_0 = h_clusphisize1frac_side1_0->GetMaximum();
        auto ymax_side1_1 = h_clusphisize1frac_side1_1->GetMaximum();
        auto ymax_side1_2 = h_clusphisize1frac_side1_2->GetMaximum();
        h_clusphisize1frac_side0_0->SetMaximum(1.2 * std::max({ymax_side0_0, ymax_side0_1, ymax_side0_2, ymax_side1_0, ymax_side1_1, ymax_side1_2}));
        h_clusphisize1frac_side0_0->Draw();
        h_clusphisize1frac_side0_1->SetMarkerColor(kBlue);
        h_clusphisize1frac_side0_1->SetLineColor(kBlue);
        h_clusphisize1frac_side0_1->Draw("same");
        h_clusphisize1frac_side0_2->SetMarkerColor(kGreen);
        h_clusphisize1frac_side0_2->SetLineColor(kGreen);
        h_clusphisize1frac_side0_2->Draw("same");
        h_clusphisize1frac_side1_0->SetMarkerColor(kViolet);
        h_clusphisize1frac_side1_0->SetLineColor(kViolet);
        h_clusphisize1frac_side1_0->Draw("same");
        h_clusphisize1frac_side1_1->SetMarkerColor(kOrange);
        h_clusphisize1frac_side1_1->SetLineColor(kOrange);
        h_clusphisize1frac_side1_1->Draw("same");
        h_clusphisize1frac_side1_2->SetMarkerColor(kBlack);
        h_clusphisize1frac_side1_2->SetLineColor(kBlack);
        h_clusphisize1frac_side1_2->Draw("same");
        
        auto legend = new TLegend(0.55, 0.7, 0.83, 0.9);
        legend->AddEntry(h_clusphisize1frac_side0_0, "Side 0, Zone 0", "pl");
        legend->AddEntry(h_clusphisize1frac_side0_1, "Side 0, Zone 1", "pl");
        legend->AddEntry(h_clusphisize1frac_side0_2, "Side 0, Zone 2", "pl");
        legend->AddEntry(h_clusphisize1frac_side1_0, "Side 1, Zone 0", "pl");
        legend->AddEntry(h_clusphisize1frac_side1_1, "Side 1, Zone 1", "pl");
        legend->AddEntry(h_clusphisize1frac_side1_2, "Side 1, Zone 2", "pl");
        legend->Draw();
    } else {
        std::cerr << "Histograms for cluster phi size not found" << std::endl;
        return;
    }

    // Update the canvas
    canvas->Update();

    std::cout << "Tpc Seeds DrawClusterInfo1() Completed" << std::endl;
}


In [17]:
%%cpp
DrawSeedClusterInfo1(qa_file_new) 

Tpc Seeds DrawClusterInfo1() Beginning
Tpc Seeds DrawClusterInfo1() Completed


In [18]:
#Tpc Cluster Info2

In [19]:
%%cpp
void TpcSeedsDrawClusterInfo2(TFile* qa_file_new) {
    std::cout << "TpcSeedsDrawClusterInfo2() Beginning" << std::endl;

    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error: Unable to open ROOT file." << std::endl;
        return;
    }

    std::string histprefix = "h_TpcSeedsQA_";
    
    // Define histograms
    TH2F* h_clusphisize1frac_pt_side[2][3];
    TH1F* h_clusphisize1frac_mean_numerator_side[2][3];
    TH1F* h_clusphisize1frac_mean_denominator_side[2][3];
    
    for (int side = 0; side < 2; side++) {
        for (int layer = 0; layer < 3; layer++) {
            h_clusphisize1frac_pt_side[side][layer] = dynamic_cast<TH2F*>(qa_file_new->Get((histprefix + Form("clusphisize1frac_pt_side%d_%d", side, layer)).c_str()));
            h_clusphisize1frac_mean_numerator_side[side][layer] = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + Form("clusphisize1frac_mean_numerator_side%d_%d", side, layer)).c_str()));
            h_clusphisize1frac_mean_denominator_side[side][layer] = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + Form("clusphisize1frac_mean_denominator_side%d_%d", side, layer)).c_str()));
        }
    }

    TCanvas* canvas = new TCanvas("track_info2", "Track Info", 1200, 900);
    canvas->Divide(4, 2);

    const char* titles[2][3] = {
        {"TPC south in", "TPC south mid", "TPC south out"},
        {"TPC north in", "TPC north mid", "TPC north out"}
    };
    
    for (int side = 0; side < 2; side++) {
        for (int layer = 0; layer < 3; layer++) {
            int padNumber = side * 3 + layer + 1;
            canvas->cd(padNumber);
            
            if (h_clusphisize1frac_pt_side[side][layer] && h_clusphisize1frac_mean_numerator_side[side][layer] && h_clusphisize1frac_mean_denominator_side[side][layer]) {
                h_clusphisize1frac_pt_side[side][layer]->SetXTitle("p_{T} [GeV]");
                h_clusphisize1frac_pt_side[side][layer]->SetYTitle("Fraction of TPC Cluster Phi Size == 1");
                h_clusphisize1frac_pt_side[side][layer]->SetZTitle("Entries");
                h_clusphisize1frac_pt_side[side][layer]->SetTitle(titles[side][layer]);
                h_clusphisize1frac_pt_side[side][layer]->Draw("colz");

                for (int bin = 1; bin <= 4; bin++) {
                    double numerator = h_clusphisize1frac_mean_numerator_side[side][layer]->GetBinContent(bin);
                    double denominator = h_clusphisize1frac_mean_denominator_side[side][layer]->GetBinContent(bin);
                    if (denominator != 0) {
                        double fraction = numerator / denominator;
                        TText* text = new TText(1.10 + 0.55 * (bin - 1), 0.90, Form("%.3f", fraction));
                        text->SetTextSize(0.06);
                        text->Draw();
                    }
                }

                gPad->SetRightMargin(0.17);
            } else {
                std::cerr << "Warning: Missing histograms for side " << side << ", layer " << layer << std::endl;
            }
        }
    }

    canvas->Update();
    std::cout << "Plotting completed" << std::endl;
}


In [20]:
%%cpp
TpcSeedsDrawClusterInfo2(qa_file_new)

TpcSeedsDrawClusterInfo2() Beginning
Plotting completed


In [21]:
#DCA Info1

In [22]:
%%cpp
void DrawDCAInfo1(TFile* qa_file_new) {
    std::cout << "Tpc Seeds DrawDCAInfo1() Beginning" << std::endl;
        if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error: Unable to open ROOT file." << std::endl;
        return;
    }

    TCanvas* canvas = new TCanvas("canvas", "DCA Info", 1000, 800);
    canvas->Divide(3, 2);

    std::string histprefix = "h_TpcSeedsQA_";
    
    std::vector<std::pair<std::string, std::string>> hist_info = {
        {"dcaxyorigin_phi_north_pos", "Positive north track DCA_{xy} wrt origin [cm]"},
        {"dcaxyorigin_phi_north_neg", "Negative north track DCA_{xy} wrt origin [cm]"},
        {"dcaxyorigin_phi_south_pos", "Positive south track DCA_{xy} wrt origin [cm]"},
        {"dcaxyorigin_phi_south_neg", "Negative south track DCA_{xy} wrt origin [cm]"},
        {"dcazorigin_phi_pos", "Positive track DCA_{z} wrt origin [cm]"},
        {"dcazorigin_phi_neg", "Negative track DCA_{z} wrt origin [cm]"}
    };
    
    for (size_t i = 0; i < hist_info.size(); ++i) {
        //TH2F* hist = dynamic_cast<TH2F*>(qa_file_new->Get(histprefix + hist_info[i].first));
        TH2F* hist = dynamic_cast<TH2F*>(qa_file_new->Get((histprefix + hist_info[i].first).c_str()));

        canvas->cd(i + 1);
        gPad->SetRightMargin(0.17);
        
        if (hist) {
            hist->SetXTitle("#phi [rad]");
            hist->SetYTitle(hist_info[i].second.c_str());
            hist->SetZTitle("Entries");
            hist->Draw("colz");
        } else {
            std::cerr << "Histogram " << hist_info[i].first << " not found" << std::endl;
        }
    }
    
    // Add text to the last pad
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.06);
    PrintRun.SetNDC();          // set to normalized coordinates
    PrintRun.SetTextAlign(23);  // center/top alignment
    
    std::ostringstream runnostream1;
    //runnostream1 << Name() << "_tpcseeds DCA Info Run " << cl->RunNumber() << ", build " << cl->build();
    
    canvas->cd(6);
    PrintRun.DrawText(0.5, 0.5, runnostream1.str().c_str());
    
    // Update the canvas
    canvas->Update();
    
    std::cout << "DrawDCAInfo1 Ending" << std::endl;
}


In [23]:
%%cpp
DrawDCAInfo1(qa_file_new)

Tpc Seeds DrawDCAInfo1() Beginning
DrawDCAInfo1 Ending


In [24]:
#DCA info2

In [25]:
%%cpp
void DrawDCAInfo2(TFile* qa_file_new) {
    std::cout << "Tpc Seeds DrawDCAInfo2() Beginning" << std::endl;
        if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error: Unable to open ROOT file." << std::endl;
        return;
    }
    
    // Create a canvas
    TCanvas* canvas = new TCanvas("canvas", "DCA Info 2", 1000, 800);
    canvas->Divide(2, 2);
    
    std::string histprefix = "h_TpcSeedsQA_";
    
    // Load histograms
    TH2F* hists[4];
    std::string histNames[] = {"dcaxyvtx_phi_pos", "dcaxyvtx_phi_neg", "dcazvtx_phi_pos", "dcazvtx_phi_neg"};
    
    for (int i = 0; i < 4; i++) {
        hists[i] = dynamic_cast<TH2F*>(qa_file_new->Get((histprefix + histNames[i]).c_str()));
    }
    
    // Titles for histograms
    std::string titles[] = {
        "Positive track DCA_{xy} wrt vertex [cm]",
        "Negative track DCA_{xy} wrt vertex [cm]",
        "Positive track DCA_{z} wrt vertex [cm]",
        "Negative track DCA_{z} wrt vertex [cm]"
    };
    
    // Draw histograms
    for (int i = 0; i < 4; i++) {
        canvas->cd(i + 1);
        gPad->SetRightMargin(0.17);
        if (hists[i]) {
            hists[i]->SetXTitle("#phi [rad]");
            hists[i]->SetYTitle(titles[i].c_str());
            hists[i]->SetZTitle("Entries");
            hists[i]->Draw("colz");
        } else {
            std::cerr << "Histogram " << histNames[i] << " not found" << std::endl;
        }
    }
    
    // Add text annotation
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.03);
    PrintRun.SetNDC();
    PrintRun.SetTextAlign(23);
    std::string runstring = "Tpc Seeds DCA Info Page 2";
    canvas->cd(0);
    PrintRun.DrawText(0.5, 1.0, runstring.c_str());
    
    // Update canvas
    canvas->Update();
    
    std::cout << "DrawDCAInfo2 Ending" << std::endl;
}


In [26]:
%%cpp
DrawDCAInfo2(qa_file_new)

Tpc Seeds DrawDCAInfo2() Beginning
DrawDCAInfo2 Ending


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas


In [27]:
# Vertex Info

In [28]:
%%cpp
void DrawVertexInfo(TFile* qa_file_new) {
    std::cout << "Tpc Seeds DrawVertexInfo() Beginning" << std::endl;

    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error: Unable to open ROOT file." << std::endl;
        return;
    }

    gROOT->SetBatch(kFALSE);  // Ensure batch mode is off to display plots

    // Check if canvas exists, if so, delete and create a new one
    TCanvas* canvas = dynamic_cast<TCanvas*>(gROOT->FindObject("canvas"));
    if (!canvas) {
        canvas = new TCanvas("canvas", "Vertex Info", 1200, 800);
    } else {
        // Clear the existing canvas
        canvas->Clear();
    }

    canvas->Divide(3, 2);

    std::string histprefix = "h_TpcSeedsQA_";

    // Retrieve histograms
    TH1F* h_nvertex = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "nrecovertices").c_str()));
    TH2F* h_vx_vy = dynamic_cast<TH2F*>(qa_file_new->Get((histprefix + "vx_vy").c_str()));
    TH1F* h_vz = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "vz").c_str()));
    TH1F* h_vt = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "vt").c_str()));
    TH1F* h_vchi2dof = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "vertexchi2dof").c_str()));
    TH1F* h_ntrackpervertex = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "ntrackspervertex").c_str()));

    // Define histograms to draw with proper draw options
    std::vector<std::pair<TH1*, std::string>> histList = {
        {h_nvertex, "Number of vertices"},
        {h_vx_vy, "Vertex X vs Y"},
        {h_vz, "Vertex Z"},
        {h_vt, "Vertex T"},
        {h_vchi2dof, "Vertex Chi2/Ndof"},
        {h_ntrackpervertex, "Tracks per Vertex"}
    };

    std::cout << "Histogram " << histprefix + "nrecovertices" << ": " << (h_nvertex ? "Found" : "Not Found") << std::endl;
    std::cout << "Histogram " << histprefix + "vx_vy" << ": " << (h_vx_vy ? "Found" : "Not Found") << std::endl;
    std::cout << "Histogram " << histprefix + "vz" << ": " << (h_vz ? "Found" : "Not Found") << std::endl;
    std::cout << "Histogram " << histprefix + "vt" << ": " << (h_vt ? "Found" : "Not Found") << std::endl;
    std::cout << "Histogram " << histprefix + "vertexchi2dof" << ": " << (h_vchi2dof ? "Found" : "Not Found") << std::endl;
    std::cout << "Histogram " << histprefix + "ntrackspervertex" << ": " << (h_ntrackpervertex ? "Found" : "Not Found") << std::endl;

    // Draw histograms
    for (size_t i = 0; i < histList.size(); i++) {
        canvas->cd(i + 1);
        gPad->SetRightMargin(0.15);
        if (histList[i].first) {
            histList[i].first->SetTitle(histList[i].second.c_str());
            if (dynamic_cast<TH2F*>(histList[i].first)) {
                histList[i].first->Draw("colz");  // 2D histograms
            } else {
                histList[i].first->Draw("hist");  // 1D histograms
            }
            gPad->Update();  // Force update for each pad
        } else {
            std::cerr << "Histogram " << histList[i].second << " not found" << std::endl;
        }
    }

    // Add text to the last pad
    canvas->cd(6);
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.06);
    PrintRun.SetNDC();         // set to normalized coordinates
    PrintRun.SetTextAlign(23); // center/top alignment
    std::string runstring1 = "Vertex Info Run";
    PrintRun.DrawText(0.5, 0.5, runstring1.c_str());

    // Update and draw canvas explicitly
    canvas->Modified();
    canvas->Update();
    canvas->Draw();

    std::cout << "DrawVertexInfo Ending" << std::endl;
}

In [29]:
%%cpp
DrawVertexInfo(qa_file_new)

Tpc Seeds DrawVertexInfo() Beginning
Histogram h_TpcSeedsQA_nrecovertices: Found
Histogram h_TpcSeedsQA_vx_vy: Found
Histogram h_TpcSeedsQA_vz: Found
Histogram h_TpcSeedsQA_vt: Found
Histogram h_TpcSeedsQA_vertexchi2dof: Found
Histogram h_TpcSeedsQA_ntrackspervertex: Found
DrawVertexInfo Ending


Error in <TUnixSystem::SetDisplay>: Can't figure out DISPLAY, set it manually
In case you run a remote ssh session, restart your ssh session with:
=========>  ssh -Y
Error in <TUnixSystem::SetDisplay>: Can't figure out DISPLAY, set it manually
In case you run a remote ssh session, restart your ssh session with:
=========>  ssh -Y


In [30]:
# dEdx Info 1

In [31]:
%%cpp 
void DrawdEdxInfo1(TFile* qa_file_new) {
    std::cout << "Tpc Seeds DrawdEdxInfo1() Beginning" << std::endl;

    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error: Unable to open ROOT file." << std::endl;
        return;
    }

    // Ensure a unique canvas name and avoid conflicts
    if (gROOT->FindObject("canvas_dEdx")) delete gROOT->FindObject("canvas_dEdx");
    TCanvas* canvas = new TCanvas("canvas_dEdx", "dEdx Info 1", 1200, 800);
    canvas->Divide(5, 2);  // Divide into 5 columns, 2 rows for 10 histograms

    std::string histprefix = "h_TpcSeedsQA_";
    TH2F* h_dedx_pq[10];

    // Retrieve histograms
    for (int i = 0; i < 10; i++) {
        h_dedx_pq[i] = dynamic_cast<TH2F*>(qa_file_new->Get(Form("%sdedx_pq_%d", histprefix.c_str(), i)));
        if (h_dedx_pq[i]) {
            std::cout << "Histogram " << Form("%sdedx_pq_%d", histprefix.c_str(), i) << ": Found" << std::endl;
        } else {
            std::cerr << "Histogram " << Form("%sdedx_pq_%d", histprefix.c_str(), i) << ": Not Found" << std::endl;
        }
    }

    // Draw histograms
    for (int i = 0; i < 10; i++) {
        if (h_dedx_pq[i]) {
            canvas->cd(i + 1);  // Select pad (1-based indexing)
            gPad->SetRightMargin(0.17);
            
            h_dedx_pq[i]->SetXTitle("p#timesq");
            h_dedx_pq[i]->SetYTitle("Mean cluster ADC corrected by path length [arb]");
            h_dedx_pq[i]->SetZTitle("Entries");
            h_dedx_pq[i]->Draw("colz");

            // Add text annotation for gz range
            int gzmin = -100 + i * 20;
            int gzmax = -100 + (i + 1) * 20;
            TPaveText* pt = new TPaveText(0.2, 0.7, 0.45, 0.9, "brNDC");
            pt->AddText(Form("Cluster gz in [%d,%d] cm", gzmin, gzmax));
            pt->SetTextAlign(12);
            pt->SetTextFont(42);
            pt->SetTextSize(0.05);
            pt->SetFillStyle(0);
            pt->SetBorderSize(0);
            pt->Draw("same");
        }
    }

    // Add run information text
    canvas->cd(10);  // Last pad for text
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.03);
    PrintRun.SetNDC();         // Use normalized coordinates
    PrintRun.SetTextAlign(23); // Center/top alignment
    PrintRun.DrawText(0.5, 1.0, "TPC Seeds dE/dx vs p*q in Z bin");

    // Update and draw the canvas
    canvas->Update();
    canvas->Draw();

    std::cout << "DrawdEdxInfo1 Ending" << std::endl;
}


In [32]:
%%cpp
DrawdEdxInfo1(qa_file_new)

Tpc Seeds DrawdEdxInfo1() Beginning
Histogram h_TpcSeedsQA_dedx_pq_0: Found
Histogram h_TpcSeedsQA_dedx_pq_1: Found
Histogram h_TpcSeedsQA_dedx_pq_2: Found
Histogram h_TpcSeedsQA_dedx_pq_3: Found
Histogram h_TpcSeedsQA_dedx_pq_4: Found
Histogram h_TpcSeedsQA_dedx_pq_5: Found
Histogram h_TpcSeedsQA_dedx_pq_6: Found
Histogram h_TpcSeedsQA_dedx_pq_7: Found
Histogram h_TpcSeedsQA_dedx_pq_8: Found
Histogram h_TpcSeedsQA_dedx_pq_9: Found
DrawdEdxInfo1 Ending


In [33]:
#dEdx Info2

In [35]:
%%cpp
int TpcSeeddEdxInfo2(TFile* qa_file_new)
{
    std::cout << "Tpc Seeds DrawdEdxInfo2() Beginning" << std::endl;

    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error: Unable to open ROOT file." << std::endl;
        return -1;  // Should return -1 in case of failure to open the file
    }

    std::string histprefix = "h_TpcSeedsQA_";

    TH2F *h_dedx_pq[10];
    for (int i = 0; i < 10; i++)
    {
        h_dedx_pq[i] = dynamic_cast<TH2F*>(qa_file_new->Get(Form("%sdedx_pq_%d", histprefix.c_str(), i)));
    }

    TH2F * h_dedx_pca = dynamic_cast<TH2F*>(qa_file_new->Get(Form("%sdedx_pcaz", histprefix.c_str())));

    // Declare canvas1 outside the conditional block
    TCanvas* canvas1 = nullptr;
    if (!gROOT->FindObject("dEdx_info2"))
    {
        // Create the canvas only if it does not already exist
        canvas1 = new TCanvas("canvas1", "dEdx_info2", 800, 600);
        canvas1->Divide(1, 3); // Divide the canvas into 3 pads
    }

    // Check if the canvas was successfully created
    if (canvas1 == nullptr)
    {
        std::cerr << "Error: Canvas creation failed!" << std::endl;
        return -1;
    }

    canvas1->cd(1); // Use canvas1 for the first pad
    if (h_dedx_pq[0] && h_dedx_pq[4])
    {
        int xBinMin = h_dedx_pq[0]->GetXaxis()->FindBin(-3.);
        int xBinMax = h_dedx_pq[0]->GetXaxis()->FindBin(0.2);

        TH1D *h_dedx_z0 = h_dedx_pq[0]->ProjectionY("h_dedx_z0", xBinMin, xBinMax);
        TH1D *h_dedx_z4 = h_dedx_pq[4]->ProjectionY("h_dedx_z4", xBinMin, xBinMax);

        h_dedx_z0->GetXaxis()->SetRangeUser(0, 3000);
        h_dedx_z4->GetXaxis()->SetRangeUser(0, 3000);

        h_dedx_z0->Scale(h_dedx_z4->Integral() / h_dedx_z0->Integral());

        float ymax = std::max(h_dedx_z0->GetMaximum(), h_dedx_z4->GetMaximum());

        h_dedx_z0->SetMaximum(1.1 * ymax);
        h_dedx_z0->SetLineColor(kRed);
        h_dedx_z0->Draw("hist");
        h_dedx_z0->GetXaxis()->SetTitle("Mean cluster Adc corrected by path length");
        h_dedx_z0->GetYaxis()->SetTitle("Events");
        h_dedx_z4->SetLineColor(kBlue);
        h_dedx_z4->Draw("hist,same");

        TF1 *landauFit_z0 = new TF1("landauFit_z0", "[2] * TMath::Landau(x,[0],[1])", 0, 3000);
        TF1 *landauFit_z4 = new TF1("landauFit_z4", "[2] * TMath::Landau(x,[0],[1])", 0, 3000);
        landauFit_z0->SetParameters(0.8 * h_dedx_z0->GetMean(), 0.5 * h_dedx_z0->GetRMS(), h_dedx_z0->Integral());
        landauFit_z4->SetParameters(0.8 * h_dedx_z4->GetMean(), 0.5 * h_dedx_z4->GetRMS(), h_dedx_z4->Integral());
        landauFit_z0->SetParLimits(0, 0.05 * h_dedx_z0->GetMean(), 2 * h_dedx_z0->GetMean());
        landauFit_z4->SetParLimits(0, 0.05 * h_dedx_z4->GetMean(), 2 * h_dedx_z4->GetMean());
        landauFit_z0->SetParLimits(1, 0.1 * h_dedx_z0->GetRMS(), 5 * h_dedx_z0->GetRMS());
        landauFit_z4->SetParLimits(1, 0.1 * h_dedx_z4->GetRMS(), 5 * h_dedx_z4->GetRMS());
        landauFit_z0->SetParLimits(2, 0, 2 * h_dedx_z0->Integral());
        landauFit_z4->SetParLimits(2, 0, 2 * h_dedx_z4->Integral());
        h_dedx_z0->Fit("landauFit_z0");
        h_dedx_z4->Fit("landauFit_z4");

        landauFit_z0->SetLineColor(kRed + 1);
        landauFit_z0->SetLineWidth(2);
        landauFit_z4->SetLineColor(kBlue + 1);
        landauFit_z4->SetLineWidth(2);

        landauFit_z0->Draw("same");
        landauFit_z4->Draw("same");

        float xpos_z0 = landauFit_z0->GetParameter(0);
        float xpos_z4 = landauFit_z4->GetParameter(0);
        float z0_to_z4_ratio = xpos_z0 / xpos_z4;
        if (std::isnan(z0_to_z4_ratio))
        {
            xpos_z0 = -1;
            xpos_z4 = -1;
        }

        TPaveText *pt = new TPaveText(0.48, 0.6, 0.9, 0.9, "brNDC");
        pt->AddText("Negative tracks p > 0.2 GeV");
        pt->AddText("Landau Fit");
        pt->AddText(Form("Mean #color[2]{HighZ} / #color[4]{LowZ} = %.2f", z0_to_z4_ratio));
        pt->SetTextAlign(12);
        pt->SetTextFont(42);
        pt->SetTextSize(0.05);
        pt->SetFillStyle(0);
        pt->SetBorderSize(0);
        pt->Draw();

        TLegend *legend = new TLegend(0.48, 0.45, 0.9, 0.6);
        legend->AddEntry(h_dedx_z0, "Cluster Z in [-100,-80] cm", "l");
        legend->AddEntry(h_dedx_z4, "Cluster Z in [-20,0] cm", "l");
        legend->SetTextSize(0.04);
        legend->Draw();

        TLine *line_z0 = new TLine(xpos_z0, 0, xpos_z0, 1.1 * ymax);
        line_z0->SetLineStyle(2);
        line_z0->SetLineWidth(2);
        line_z0->SetLineColor(kRed);
        line_z0->Draw();

        TLine *line_z4 = new TLine(xpos_z4, 0, xpos_z4, 1.1 * ymax);
        line_z4->SetLineStyle(2);
        line_z4->SetLineWidth(2);
        line_z4->SetLineColor(kBlue);
        line_z4->Draw();
    }

    // Now use canvas1 for the second pad
    canvas1->cd(2);
    if (h_dedx_pq[9] && h_dedx_pq[5])
    {
        int xBinMin = h_dedx_pq[9]->GetXaxis()->FindBin(-3.);
        int xBinMax = h_dedx_pq[9]->GetXaxis()->FindBin(0.2);

        TH1D *h_dedx_z9 = h_dedx_pq[9]->ProjectionY("h_dedx_z9", xBinMin, xBinMax);
        TH1D *h_dedx_z5 = h_dedx_pq[5]->ProjectionY("h_dedx_z5", xBinMin, xBinMax);

        h_dedx_z9->GetXaxis()->SetRangeUser(0, 3000);
        h_dedx_z5->GetXaxis()->SetRangeUser(0, 3000);

        h_dedx_z9->Scale(h_dedx_z5->Integral() / h_dedx_z9->Integral());

        float ymax = std::max(h_dedx_z9->GetMaximum(), h_dedx_z5->GetMaximum());

        h_dedx_z9->SetMaximum(1.1 * ymax);
        h_dedx_z9->SetLineColor(kRed);
        h_dedx_z9->Draw("hist");
        h_dedx_z9->GetXaxis()->SetTitle("Mean cluster Adc corrected by path length");
        h_dedx_z9->GetYaxis()->SetTitle("Events");
        h_dedx_z5->SetLineColor(kBlue);
        h_dedx_z5->Draw("hist,same");

        TF1 *landauFit_z9 = new TF1("landauFit_z9", "[2] * TMath::Landau(x,[0],[1])", 0, 3000);
        TF1 *landauFit_z5 = new TF1("landauFit_z5", "[2] * TMath::Landau(x,[0],[1])", 0, 3000);
        landauFit_z9->SetParameters(0.8 * h_dedx_z9->GetMean(), 0.5 * h_dedx_z9->GetRMS(), h_dedx_z9->Integral());
        landauFit_z5->SetParameters(0.8 * h_dedx_z5->GetMean(), 0.5 * h_dedx_z5->GetRMS(), h_dedx_z5->Integral());
        landauFit_z9->SetParLimits(0, 0.05 * h_dedx_z9->GetMean(), 2 * h_dedx_z9->GetMean());
        landauFit_z5->SetParLimits(0, 0.05 * h_dedx_z5->GetMean(), 2 * h_dedx_z5->GetMean());
        landauFit_z9->SetParLimits(1, 0.1 * h_dedx_z9->GetRMS(), 5 * h_dedx_z9->GetRMS());
        landauFit_z5->SetParLimits(1, 0.1 * h_dedx_z5->GetRMS(), 5 * h_dedx_z5->GetRMS());
        landauFit_z9->SetParLimits(2, 0, 2 * h_dedx_z9->Integral());
        landauFit_z5->SetParLimits(2, 0, 2 * h_dedx_z5->Integral());
        h_dedx_z9->Fit("landauFit_z9");
        h_dedx_z5->Fit("landauFit_z5");

        landauFit_z9->SetLineColor(kRed + 1);
        landauFit_z9->SetLineWidth(2);
        landauFit_z5->SetLineColor(kBlue + 1);
        landauFit_z5->SetLineWidth(2);

        landauFit_z9->Draw("same");
        landauFit_z5->Draw("same");

        float xpos_z9 = landauFit_z9->GetParameter(0);
        float xpos_z5 = landauFit_z5->GetParameter(0);
        float z9_to_z5_ratio = xpos_z9 / xpos_z5;
        if (std::isnan(z9_to_z5_ratio))
        {
            xpos_z9 = -1;
            xpos_z5 = -1;
        }

        TPaveText *pt1 = new TPaveText(0.48, 0.6, 0.9, 0.9, "brNDC");
        pt1->AddText("Positive tracks p > 0.2 GeV");
        pt1->AddText("Landau Fit");
        pt1->AddText(Form("Mean #color[2]{HighZ} / #color[4]{LowZ} = %.2f", z9_to_z5_ratio));
        pt1->SetTextAlign(12);
        pt1->SetTextFont(42);
        pt1->SetTextSize(0.05);
        pt1->SetFillStyle(0);
        pt1->SetBorderSize(0);
        pt1->Draw();

        TLegend *legend1 = new TLegend(0.48, 0.45, 0.9, 0.6);
        legend1->AddEntry(h_dedx_z9, "Cluster Z in [-100,-80] cm", "l");
        legend1->AddEntry(h_dedx_z5, "Cluster Z in [-20,0] cm", "l");
        legend1->SetTextSize(0.04);
        legend1->Draw();

        TLine *line_z9 = new TLine(xpos_z9, 0, xpos_z9, 1.1 * ymax);
        line_z9->SetLineStyle(2);
        line_z9->SetLineWidth(2);
        line_z9->SetLineColor(kRed);
        line_z9->Draw();

        TLine *line_z5 = new TLine(xpos_z5, 0, xpos_z5, 1.1 * ymax);
        line_z5->SetLineStyle(2);
        line_z5->SetLineWidth(2);
        line_z5->SetLineColor(kBlue);
        line_z5->Draw();
    }

    // Now use canvas1 for the third pad
    canvas1->cd(3);
    h_dedx_pca->Draw("colz");

    canvas1->Update();

    return 0;
}


In [36]:
%%cpp
TpcSeeddEdxInfo2(qa_file_new)

Tpc Seeds DrawdEdxInfo2() Beginning
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      28.4078
NDf                       =           15
Edm                       =  2.49743e-07
NCalls                    =          101
p0                        =      810.272   +/-   115.109      	 (limited)
p1                        =      342.492   +/-   92.8509      	 (limited)
p2                        =      60.9833   +/-   11.4513      	 (limited)
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      37.5135
NDf                       =           20
Edm                       =  4.67684e-09
NCalls                    =          114
p0                        =      618.122   +/-   17.8953      	 (limited)
p1                        =       98.857   +/-   12.9033      	 (limited)
p2                        =      167.484   +/-   24.1886      	 (limited)
****************************************
Minimizer